## CSS Project Part 2
Arathy Pillai, Alexander Guth, Hammad Akthar, Hiba Saleel and Julian Jetz

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from string import punctuation

In [2]:
plot_size = plt.rcParams["figure.figsize"]  
plot_size[0] = 15  
plot_size[1] = 8  
plt.rcParams["figure.figsize"] = plot_size  

### Load data and determine na values
repost and Text are determined as na values, because they ... **[TODO]**

In [3]:
na_values = ['nan', 'N/A', 'NaN', 'NaT', '', 'Text', 'repost', 'r e p o s t -', 'Repost']
#na_values = ['nan', 'N/A', 'NaN', 'NaT', '0', '', 'Text', 'repost', 'r e p o s t -', 'Repost']
ira_data = pd.read_csv('../Data/data_IRA_Ads.csv', sep=";", parse_dates=['AD_CREATION_DATE', 'AD_END_DATE'], 
                       dayfirst=False, index_col='AD_ID', na_values=na_values)
ira_data = ira_data[ira_data['AD_IMPRESSIONS']>0]

### Click per View Ratio
#Clicks / #Impressions

In [4]:
ira_data.insert(5, 'AD_CLICKS_PER_VIEW', ira_data['AD_CLICKS']/ira_data['AD_IMPRESSIONS'])

In [5]:
ira_data['AD_CREATION_WEEKDAY'] = ira_data['AD_CREATION_DATE'].dt.day_name()
ira_data['AD_CREATION_YEAR'] = ira_data['AD_CREATION_DATE'].dt.year
ira_data['AD_CREATION_MONTH'] = ira_data['AD_CREATION_DATE'].dt.month
ira_data['AD_CREATION_DAY'] = ira_data['AD_CREATION_DATE'].dt.day
ira_data['AD_CREATION_TIME'] = ira_data['AD_CREATION_DATE'].dt.time

### Convert ruble
Removes "RUB" from the cell AD_SPEND

In [6]:
ira_data['AD_SPEND']=ira_data['AD_SPEND'].str.replace(' RUB','')

### Drop all subsets, where AD_TEXT is na

In [7]:
ira_data=ira_data.dropna(subset=['AD_TEXT']);
ira_data['AD_TEXT'].count()

2533

### Remove hyperlinks

In [8]:
ira_data['AD_TEXT']=ira_data['AD_TEXT'].str.replace(r'https?:\/\/.*[\r\n]*', '')
ira_data['AD_TEXT']=ira_data['AD_TEXT'].str.replace(r'http?:\/\/.*[\r\n]*', '')

### Select all duplicate ad texts

In [9]:
duplicateDF = pd.concat(g for _, g in ira_data.groupby("AD_TEXT") if len(g) > 1).copy()
duplicateDF.sort_values(by=['AD_TEXT'], inplace=True)

In [10]:
ira_data['AD_TEXT'].count()

2533

In [11]:
uniqueDF = ira_data.copy()
uniqueDF.drop_duplicates(subset ='AD_TEXT', 
                     keep = False, inplace = True)

### Total Duplicates

In [12]:
ira_data['AD_TEXT'].count()

2533

In [13]:
duplicateDF['AD_TEXT'].count()

614

In [14]:
uniqueDF['AD_TEXT'].count()

1919

### Unique Duplicate Pairs

In [15]:
duplicateDF['AD_TEXT'].nunique()

203

### Occurence of Duplicate Pairs

In [16]:
duplicateDF['AD_TEXT'].value_counts().values

array([13, 12,  9,  8,  7,  7,  6,  6,  6,  6,  6,  6,  6,  6,  5,  5,  5,
        5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  4,  4,
        4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  3,  3,  3,  3,  3,
        3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
        3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
        3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2])

In [17]:
duplicateDF[duplicateDF['AD_IMPRESSIONS']>0]['AD_TEXT'].value_counts()

We speak for all fellow members of LGBT community across the nation.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     13
We are proud to be black and

In [18]:
duplicateDF[(duplicateDF['AD_IMPRESSIONS']>0) & duplicateDF['AD_TEXT'].str.match('We speak for all fellow members of LGBT community across the nation.')][['AD_CLICKS', 'AD_IMPRESSIONS', 'AD_CREATION_DATE', 'AD_CREATION_WEEKDAY','AD_CREATION_YEAR', 'AD_CREATION_MONTH','AD_CREATION_DAY','AD_CREATION_TIME','AD_TEXT', 'AD_SPEND', 'AD_POTENTIAL_TARGET']].head(20)

,AD_CLICKS,AD_IMPRESSIONS,AD_CREATION_DATE,AD_CREATION_WEEKDAY,AD_CREATION_YEAR,AD_CREATION_MONTH,AD_CREATION_DAY,AD_CREATION_TIME,AD_TEXT,AD_SPEND,AD_POTENTIAL_TARGET
AD_ID,,,,,,,,,,,
669,91,1997,2015-06-17 03:15:18,Wednesday,2015,6,17,03:15:18,We speak for all fellow members of LGBT commun...,2673.94,144000000.0
591,5441,93925,2016-03-23 05:36:11,Wednesday,2016,3,23,05:36:11,We speak for all fellow members of LGBT commun...,31713.12,NaN
590,6566,67221,2015-11-05 23:11:58,Thursday,2015,11,5,23:11:58,We speak for all fellow members of LGBT commun...,46994.76,36000000.0
588,21449,175460,2015-07-14 00:01:38,Tuesday,2015,7,14,00:01:38,We speak for all fellow members of LGBT commun...,93754.38,41000000.0
587,542,15175,2015-07-09 01:50:28,Thursday,2015,7,9,01:50:28,We speak for all fellow members of LGBT commun...,6201.70,35000000.0
586,59,1811,2015-07-07 01:51:07,Tuesday,2015,7,7,01:51:07,We speak for all fellow members of LGBT commun...,757.65,166000000.0
592,7289,190076,2016-08-04 04:20:05,Thursday,2016,8,4,04:20:05,We speak for all fellow members of LGBT commun...,36000.00,NaN
584,3206,31618,2015-06-23 05:06:57,Tuesday,2015,6,23,05:06:57,We speak for all fellow members of LGBT commun...,16630.20,143000000.0
583,186,4265,2015-06-22 08:19:07,Monday,2015,6,22,08:19:07,We speak for all fellow members of LGBT commun...,1393.55,139000000.0


In [19]:
duplicateDF.sample(25)

,AD_CLICKS,AD_CREATION_DATE,AD_END_DATE,AD_IMPRESSIONS,AD_LANDING_PAGE,AD_CLICKS_PER_VIEW,AD_PATH,AD_SPEND,AD_TARGETING_AGE,AD_TARGETING_CUSTOM_AUDIENCE,...,AD_TARGETING_LOCATION_RECENTLY_IN,AD_TARGETING_PEOPLE_WHO_MATCH,AD_TARGETING_PLACEMENTS,AD_TEXT,AD_POTENTIAL_TARGET,AD_CREATION_WEEKDAY,AD_CREATION_YEAR,AD_CREATION_MONTH,AD_CREATION_DAY,AD_CREATION_TIME
AD_ID,,,,,,,,,,,,,,,,,,,,,
2308,40,2016-09-28 02:04:24,2016-09-29 05:34:45,284,https://blackmattersus.com/16599-institutional...,0.140845,2016-q3/2016-09/P(1)0004638.pdf,137.71,18 - 65+,NaN,...,NaN,NaN,Instagram Feed or News Feed on mobile devices,Institutional racism takes a hold of our schoo...,6600000.0,Wednesday,2016,9,28,02:04:24
1271,215,2016-04-13 07:48:07,NaT,1737,https://www.facebook.com/stop-al-896610653786585/,0.123777,2016-q2/2016-04/P(1)0003145.pdf,835.72,35 - 65+,NaN,...,NaN,NaN,News Feed on desktop computers or News Feed on...,"If you want to save America, you're a part of ...",27000000.0,Wednesday,2016,4,13,07:48:07
2815,17214,2016-10-18 04:53:49,NaT,307235,https://www.facebook.com/woke-blacks-294234600...,0.056029,2016-q4/2016-10/P(1)0005366.pdf,23263.09,18 - 65+,NaN,...,NaN,NaN,"News Feed on desktop computers, News Feed on m...","Staying Woke,building-up our Black communities...",2800000.0,Tuesday,2016,10,18,04:53:49
874,15,2016-04-13 04:44:13,NaT,697,https://www.facebook.com/villiamsandkaivin,0.021521,2016-q2/2016-04/P(1)0006246.pdf,338.88,18 - 45,NaN,...,NaN,NaN,News Feed on desktop computers or News Feed on...,It's time to wake up and break our chains! Tog...,17000000.0,Wednesday,2016,4,13,04:44:13
2819,7,2016-10-17 08:49:28,NaT,89,https://www.facebook.com/woke-blacks-294234600...,0.078652,2016-q4/2016-10/P(1)0005374.pdf,133.17,18 - 65+,NaN,...,NaN,NaN,"News Feed on desktop computers, News Feed on m...","Come check out our page, Uniting Black people ...",24000000.0,Monday,2016,10,17,08:49:28
1303,55,2015-12-22 06:40:56,NaT,486,https://www.facebook.com/stoptherefugees/,0.113169,2015-q4/2015-12/P(1)0003201.pdf,712.21,18 - 65+,NaN,...,NaN,NaN,News Feed on desktop computers or News Feed on...,Stop refugees! The're taking our jobs!,5200000.0,Tuesday,2015,12,22,06:40:56
534,0,2016-04-13 03:24:02,NaT,5,https://www.facebook.com/events/i678907209017708/,0.000000,2016-q2/2016-04/P(1)0001235.pdf,10.68,18 - 59,NaN,...,NaN,NaN,News Feed on desktop computers or News Feed on...,Stand for the people not the Government! Join ...,NaN,Wednesday,2016,4,13,03:24:02
1781,2,2016-04-12 00:23:03,2016-04-19 07:58:26,51,https://www.facebook.com/memopolis-45047461515...,0.039216,2016-q2/2016-04/P(1)0003900.pdf,68.98,13 - 65+,NaN,...,NaN,NaN,News Feed on mobile devices,Welcome to the city of memes - Memopolis.,230000000.0,Tuesday,2016,4,12,00:23:03
855,0,2016-01-14 06:38:38,NaT,48,https://www.facebook.com/villiamsandkaivin/,0.000000,2016-q1/2016-01/P(1)0006242.pdf,NaN,18 - 45,NaN,...,NaN,NaN,News Feed on desktop computers or News Feed on...,Your voice counts! Here you can find news and ...,7700000.0,Thursday,2016,1,14,06:38:38


In [20]:
uniqueDF.sample(25)

,AD_CLICKS,AD_CREATION_DATE,AD_END_DATE,AD_IMPRESSIONS,AD_LANDING_PAGE,AD_CLICKS_PER_VIEW,AD_PATH,AD_SPEND,AD_TARGETING_AGE,AD_TARGETING_CUSTOM_AUDIENCE,...,AD_TARGETING_LOCATION_RECENTLY_IN,AD_TARGETING_PEOPLE_WHO_MATCH,AD_TARGETING_PLACEMENTS,AD_TEXT,AD_POTENTIAL_TARGET,AD_CREATION_WEEKDAY,AD_CREATION_YEAR,AD_CREATION_MONTH,AD_CREATION_DAY,AD_CREATION_TIME
AD_ID,,,,,,,,,,,,,,,,,,,,,
2272,500,2016-09-30 04:43:26,2016-10-03 04:43:25,3689,https://www.facebook.com/blackmattersus/.mvmnt/,0.135538,2016-q3/2016-09/P(1)0004578.pdf,300.00,16 - 65+,NaN,...,NaN,NaN,News Feed on desktop computers or News Feed on...,Do everything in your power to change the soci...,3000000.0,Friday,2016,9,30,04:43:26
1250,21,2015-10-08 00:39:23,2015-10-09 23:59:59,19765,https://www.instagram.com/instotus/,0.001062,2015-q4/2015-10/P(1)0003113.pdf,478.68,18 - 65+,NaN,...,NaN,NaN,Instagram Feed,We are honored to be the main and the biggest ...,115000000.0,Thursday,2015,10,8,00:39:23
1262,615,2017-05-23 04:36:29,2017-05-24 04:36:29,5887,https://www.facebook.com/muslimamerica/,0.104467,2017-05/P(1)0003131.pdf,999.80,18 - 65+,NaN,...,NaN,NaN,News Feed on desktop computers or News Feed on...,This is a call for love. This is a call for me...,NaN,Tuesday,2017,5,23,04:36:29
1069,38,2015-09-11 05:26:38,NaT,595,https://www.facebook.com/blacktivist-128371547...,0.063866,2015-q3/2015-09/P(1)0002875.pdf,876.04,18 - 55,NaN,...,NaN,NaN,News Feed on desktop computers or News Feed on...,Gone but never forgotten! Join to remember fal...,NaN,Friday,2015,9,11,05:26:38
2721,2029,2017-02-16 00:41:10,2017-02-17 00:41:10,13281,https://www.facebook.com/brownunitedfront/,0.152775,2017-q1/2017-02/P(1)0000971.pdf,200.00,18 - 65+,NaN,...,NaN,NaN,News Feed on desktop computers or News Feed on...,"Mictlantecuhtli, the Aztec god of death",49000000.0,Thursday,2017,2,16,00:41:10
2733,1,2017-03-16 08:08:39,2017-03-17 08:08:39,4,https://www.facebook.com/brownunitedfront/,0.250000,2017-q1/2017-03/P(1)0000985.pdf,0.14,18 - 65+,NaN,...,NaN,NaN,News Feed on desktop computers or News Feed on...,I will just leave it here,49000000.0,Thursday,2017,3,16,08:08:39
1818,285,2016-05-18 04:23:14,2016-05-19 04:23:13,3625,https://www.facebook.com/memopolis-45047461515...,0.078621,2016-q2/2016-05/P(1)0003950.pdf,500.00,18 - 65+,NaN,...,NaN,NaN,News Feed on desktop computers or News Feed on...,Can you smrell what the wok is cookin,59000000.0,Wednesday,2016,5,18,04:23:14
1869,71,2016-03-23 06:29:16,NaT,15109,https://www.instagram.com/cop_block_us/,0.004699,2016-q1/2016-03/P(1)0000569.pdf,3977.37,13 - 45,NaN,...,NaN,NaN,Instagram Feed,Cheers to the actual heroes that protect and s...,490000.0,Wednesday,2016,3,23,06:29:16
1702,2549,2016-07-13 01:11:32,2016-07-14 01:11:31,9900,https://www.facebook.com/memopolis-45047461515...,0.257475,2016-q3/2016-07/P(1)0003790.pdf,500.00,14 - 40,NaN,...,NaN,NaN,News Feed on desktop computers or News Feed on...,Pulls out a rock and throws it at the cop to t...,36000000.0,Wednesday,2016,7,13,01:11:32
